![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [10]:
# Start your code here!
import pandas as pd
#load the dataset
ds_jobs = pd.read_csv("Dataset/customer_train.csv")
ds_jobs.head()
ds_jobs_clean= ds_jobs.copy()

In [11]:
ds_jobs_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

#### We observe the memory usage parameter in the dataset and we se that it uses 2.0+ MB of memory. We need to ensure that the data in the dataframe is stored as efficiently as possible so as to ensure that modelling is also done efficiently 

In [12]:
# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'relevant_experience': ['No relevant experience', 'Has relevant experience'],
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

for col in ds_jobs_clean:
    #convert the columns containtng ordered categorical data  to  the ordered categories
    if col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
        ds_jobs_clean[col] = ds_jobs_clean[col].astype(category)
    #convert the int columns to int32
    elif ds_jobs[col].dtype == "int":
         ds_jobs_clean[col] = ds_jobs_clean[col].astype("int32")
    #convert the float colunms to float16        
    elif ds_jobs_clean[col].dtype == "float":
        ds_jobs_clean[col] = ds_jobs_clean[col].astype("float16")
    #convert the remaining columns into categorical data
    else:   
        ds_jobs_clean[col] = ds_jobs_clean[col].astype("category")
ds_jobs_clean.info()        


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  category
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  category
 13  job_change              19158 non-null  category
dtypes: category(13), float

#### After modifying th edata types of the several columns we can now see that the memory usage parameter is now close to 1MB which is relatively smaller than what we originally had make it slightly efficient for when modelling of the data is done.

In [13]:
# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]
ds_jobs_clean

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19097,25447,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,>4,57,0
19101,6803,city_16,0.910156,Male,Has relevant experience,no_enrollment,High School,NaN,10,10000+,Pvt Ltd,1,89,0
19103,32932,city_10,0.895020,Male,Has relevant experience,Part time course,Masters,Other,>20,1000-4999,Pvt Ltd,>4,18,0
19128,3365,city_16,0.910156,NaN,Has relevant experience,no_enrollment,Graduate,Humanities,>20,1000-4999,Pvt Ltd,>4,23,0
